<img style="float: right;" src="images/spock.png" width="500">

# spock


In [11]:
import numpy as np
import navicat_spock
from ipywidgets import interactive, widgets, HTML, Button
from typing import Callable, Any
import warnings
import os
import markdown
import codecs
import pickle
import pandas as pd
from IPython.display import display, FileLink, clear_output
from contextlib import contextmanager
from io import StringIO 
import sys


warnings.simplefilter('ignore')
html_description = markdown.markdown("""
cell2mol is able to analyze cif files containing crystallographic unit cell information and provide a chemical interpretation of the individual molecular species. Run the entire notebook and use the widget to proceed through the four tabs sequentially.""")
description=HTML(html_description)

pre_accordion = widgets.Accordion()
pre_accordion.children = [description]
pre_accordion.set_title(0, 'Description')
pre_accordion.selected_index = None
pre_accordion

Accordion(children=(HTML(value='<p>cell2mol is able to analyze cif files containing crystallographic unit cell…

In [12]:
# Import the core functionalities of cell2mol
from navicat_spock.spock import run_spock_from_args

# Define some basic functionalities

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout


class ExecutionButton(Button):
    def __init__(self, action, WidgetOut, **kwargs):
        super(ExecutionButton, self).__init__(**kwargs)
        self.action = action
        self.tooltip='Click me to run'
        self.description='Run'
        self.icon='fa-check'
        self.output = WidgetOut
        self.on_click(self.__on_click)
        
    @contextmanager
    def show_loading(self):
        self.description = 'Running...'
        self.icon = 'fa-stop'
        yield
        self.description = 'Ready to run again'
        self.icon = 'fa-check'
           
    def __on_click(self, b):
        with self.show_loading():
            toprint = self.action(run=True, out=self.output)
        with self.output :
            for line in toprint:
                print(line)

#############################

Widget1UploadConfiguration = widgets.FileUpload(
 accept='.csv',  # Accepted file extension e.g. '.cif'
 multiple=False,  # True to accept multiple files upload else False
 wait=True
)

examples = ('Sabatier volcano', 'test/sabatier.csv'),
widget_choose_filename = widgets.Dropdown(options=examples,description='Data:')

Widget1Out = widgets.Output()
def chooseAndPlotConfigurationAltOutput(filename):
    Widget1Out.clear_output()
    #with Widget1Out:
    #    plot_from_file(filename)
        
Widget1ExampleConfiguration = interactive(chooseAndPlotConfigurationAltOutput, filename=widget_choose_filename)

# This function updates the dropdown list when a new file is uploaded
def updateWidget1ExampleConfiguration(*args):
    uploaded = next(iter(Widget1UploadConfiguration.value))
    cif_content = codecs.decode(uploaded['content'], encoding="utf-8")
    print(cif_content)
    cif_name = uploaded['name']
    mypath="./uploaded/"
    with open(mypath + cif_name, 'w') as f: 
        f.write(cif_content)
    found_files=[]
    for f in os.listdir(mypath):
        found_files.append(f)
    all_files = found_files
    widget_choose_filename.options = all_files
    

Widget1UploadConfiguration.observe(updateWidget1ExampleConfiguration,names='value')
Widget1ExamplesAndUpload = widgets.HBox([Widget1ExampleConfiguration,Widget1UploadConfiguration])
Widget1Text = HTML(markdown.markdown("""
Choose a data sheet from the examples or upload your own in .csv format.
"""))
Widget1 = widgets.VBox([Widget1Text,Widget1ExamplesAndUpload,Widget1Out])

Widget2Text = HTML(markdown.markdown("""
Will run spock on the csv provided. When finished, output will be printed below.
"""))
Widget2BottomText = HTML(markdown.markdown("""
Output:
"""))
Widget2Out = widgets.Output(layout={'border': '1px solid black'})

def run_spock(run=False, out=None):
    if run :
        csv_name = widget_choose_filename.value
        if "/" in csv_name :
            csv_name = csv_name.split("/")[-1]
        err_name = csv_name.replace('.csv', '.err')
        input_path = "./uploaded/" + csv_name
        with Capturing() as output:
            run_spock_from_args(pd.read_csv(input_path))
        error = False
        with open(error_path, 'r') as err:
            for line in err.readlines():
                if "Error" in line:
                    output.extend([line])
                    error = True
        if error :
            output.extend([f"Parsing of .cif file {cif_name} failed due to the error above."])
        else :
            output.extend([f"Infofile {info_name} generated from {cif_name} succesfully."])
        return output
    
def clear (run=False, out=None, *some):
    if run :
        with out:
            output=[]
            for i in some:
                i.clear_output() 
            output.extend(["All the output was cleared"])
    return output

Widget2 = widgets.VBox([Widget2Text, ExecutionButton(run_spock, Widget2Out), Widget2BottomText, Widget2Out])          

#############################
# Put widget in tab
#############################

children = [Widget1, Widget2]
# children = [Widget1, Widget2, Widget3, Widget4, Widget5, Widget6]
tab = widgets.Tab() #layout=widgets.Layout(width='800px', height='800px'))
tab.children = children
tab.set_title(0, 'Upload file')
tab.set_title(1, 'Run spock')

In [13]:
display(tab)

FileNotFoundError: [Errno 2] No such file or directory: './uploaded/sabatier.csv'

['All the output was cleared']

In [4]:
html_acknowledgments = markdown.markdown("""
* Funding from the NCCR Catalysis funded by the SNSF.
""")
acknowledgments=widgets.VBox([
    HTML(html_acknowledgments)
])

html_howtocite = markdown.markdown("""
Please cite the spock paper~!
""")
howtocite=widgets.VBox([
    HTML(html_howtocite)
])

post_children=[acknowledgments,howtocite]
post_accordion = widgets.Accordion(layout=widgets.Layout())
#print(post_accordion.layout.keys)
post_accordion.children = post_children
post_accordion.set_title(0, 'Acknowledgments')
post_accordion.set_title(1, 'How to cite')
post_accordion.selected_index = None
post_accordion

Accordion(children=(VBox(children=(HTML(value='<ul>\n<li>Funding from the NCCR Catalysis funded by the SNSF.</…

See the project on [GitHub](https://github.com/lcmd-epfl/cell2mol)

spock, Copyright © 2024 